<a href="https://colab.research.google.com/github/ErnestoAvedillo/AAM-math/blob/main/Reto_AAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cargamos las librerias

In [301]:
#!pip install --upgrade gspread

In [302]:
#!pip install --upgrade pandas
#!pip install --upgrade Lib/datetime.py

In [303]:
!pip install bar_chart_race

In [304]:
import pandas as pd
import numpy as np
from google.colab import auth
from gspread.models import Cell
import gspread
from google.auth import default#autenticating to google
import datetime as dt
from datetime import timedelta
import matplotlib.pyplot as plt
from IPython.display import clear_output


In [305]:
gspread.__version__

'3.4.2'

In [306]:
pd.__version__

'1.5.3'

#Reading Excel AAM data.gsheet


In [307]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [308]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
file_path = "/content/drive/MyDrive/Colab Notebooks/AAM data.gsheet"
try:
    #https://docs.google.com/spreadsheets/d/179giTQNZghst9cpCtAzmTpHt0DrOtRQiBplU6-_XiUU/edit?usp=drive_link
    AAM_workbook = gc.open_by_key('179giTQNZghst9cpCtAzmTpHt0DrOtRQiBplU6-_XiUU')
    #worksheet = gc.open(file_path)
    print("Google Sheet opened successfully.")
except gspread.exceptions.SpreadsheetNotFound:
    print("Error: Spreadsheet not found.") #Convert to a DataFrame


Google Sheet opened successfully.


# **Primera parte**

Modelado de la producción

Cálculo del coste de las demandas internas.

known_references = nḿero de referencias (n).

REF = diccionario con las referencias de la empresa (TODAS , las de compra , internas  y de producto acabado.)

DC = Demanda de cliente

BOM = es una tabla de n x n con el BOM de los producto (Todos) Build of Materials

##Obteniendo un listado de las referencias.


In [309]:
BOM_worksheet = AAM_workbook.worksheet('BOM')#Convert to a DataFrame
rows = BOM_worksheet.get_all_values()#Convert to a DataFrame
df = pd.DataFrame(rows)
#creating columns name
df.columns = df.iloc[0]
df = df.iloc[1:]
REF_df = pd.concat([df['Ref Entrada'],df['Ref Salida']]).drop_duplicates()
#REF = REF_df[:].to_dict(orient='dict') does not work will try again
REF = {}
n = 0
for i in REF_df:
  REF[i] = n
  n += 1
print ("Diccionario de referencias con sus índices")
print(REF)

Diccionario de referencias con sus índices
{'AAA': 0, 'AMA': 1, 'BAJ': 2, 'AAB': 3, 'CAT': 4, 'AAE': 5, 'BAC': 6, 'CAR': 7, 'AAF': 8, 'BAD': 9, 'CAV': 10, 'AAC': 11, 'CAB': 12, 'CAX': 13, 'CAH': 14}


In [310]:
# Obtengo el numero de referencias
known_references = REF_df.shape[0]
print("Número total de referencias ", known_references)

Número total de referencias  15


## Creando el BOM que aparece en el Excel

In [311]:
#BILL OF MATERIALS
BOM = np.zeros((known_references,known_references))
for index, row in df.iterrows():
  BOM[REF[row["Ref Salida"]],REF[row["Ref Entrada"]]] = float(row["Cantidad"].replace(',', '.'))
BOM_df = pd.DataFrame(BOM,index=REF_df, columns=REF_df)
print (BOM_df)

     AAA   AMA  BAJ  AAB  CAT  AAE  BAC  CAR  AAF  BAD  CAV  AAC  CAB  CAX  \
AAA  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
AMA  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
BAJ  1.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
AAB  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
CAT  0.0  0.07  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
AAE  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
BAC  0.0  0.00  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
CAR  0.0  0.06  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
AAF  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
BAD  0.0  0.00  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
CAV  0.0  0.05  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
AAC  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [312]:
BOM = BOM_df.values
mask_compras = np.expand_dims(BOM.sum(axis=1) == 0, axis = 0)
mask_cliente = np.expand_dims(BOM.sum(axis=0) == 0, axis = 0)

## Obtención de la demanda de cliente

In [313]:
DC_worksheet = AAM_workbook.worksheet('DC') #Convert to a DataFrame
rows = DC_worksheet.get_all_values()#Convert to a DataFrame
DC_df = pd.DataFrame(rows)
DC_df.iloc[0][0] = None
# Set row 1 as column names
DC_df.columns = DC_df.iloc[0]

# Set column 1 as row names
DC_df = DC_df.set_index(DC_df.iloc[:, 0])

# Remove row 1 and column 1
DC_df = DC_df.iloc[1:, 1:]
print(DC_df)
#Demandas de cliente
Profundidad = 15
DC = np.zeros((known_references,Profundidad))
for index, row in DC_df.iterrows():
  DC[REF[index],0] = row["Total"]
print('\n')
print (DC)

0   Total Piezas/caja
CAB   560            
CAX   300            
CAH   320            


[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0

##Descargamos los datos de almacen las referencias



In [314]:
Almacen_worksheet = AAM_workbook.worksheet('ALMACEN') #Convert to a DataFrame
rows = Almacen_worksheet.get_all_values()#Convert to a DataFrame
Almacen_df = pd.DataFrame(rows)
Almacen_df.iloc[0][0] = None
# Set row 1 as column names
Almacen_df.columns = Almacen_df.iloc[0]

# Set column 1 as row names
Almacen_df = Almacen_df.set_index(Almacen_df.iloc[:, 0])

# Remove row 1 and column 1
Almacen_df = Almacen_df.iloc[1:, 1:]
print(Almacen_df)
#Demandas de cliente

0   STOCK   Piezas caja €/UD Stock minimo Stock maximo Stock stop Lote max  \
AAA  2000            16   15          240          240        240     1000   
AAB  2000            20   23          240          240        240     1000   
AAC  2000            18   50          240          240        240     1000   
AAE  2000            18   44          240          240        240     1000   
AAF  2000            16   15          240          240        240     1000   
AMA  2000            20   98          240          240        240     1000   
BAC    40            18   44           88          220        352      720   
BAD     0            32   15           30           75        120      704   
BAJ    48            16   98          196          490        784      720   
CAR     0            16   71          142          355        568      720   
CAT    32            20   30           60          150        240      720   
CAV    36            60   33           66          165        26

In [315]:
Almacen = np.zeros((known_references, 9))
for index, row in Almacen_df.iterrows():
  Almacen[REF[index],0] = row["STOCK"]
  Almacen[REF[index],1] = row["Piezas caja"]
  Almacen[REF[index],2] = row["€/UD"]
  Almacen[REF[index],3] = row["Stock minimo"]
  Almacen[REF[index],4] = row["Stock maximo"]
  Almacen[REF[index],5] = row["Stock stop"]
  Almacen[REF[index],6] = row["Lote max"]
  Almacen[REF[index],7] = row["Lote min"]
  Almacen[REF[index],8] = row["T.CICLO"]
  Almacen[REF[index],9] = row["TIPO"]
print(Almacen)

[[2000.   16.   15.  240.  240.  240. 1000.    0.    0.]
 [2000.   20.   98.  240.  240.  240. 1000.    0.    0.]
 [  48.   16.   98.  196.  490.  784.  720.    0.   55.]
 [2000.   20.   23.  240.  240.  240. 1000.    0.    0.]
 [  32.   20.   30.   60.  150.  240.  720.    0.   30.]
 [2000.   18.   44.  240.  240.  240. 1000.    0.    0.]
 [  40.   18.   44.   88.  220.  352.  720.    0.   50.]
 [   0.   16.   71.  142.  355.  568.  720.    0.   40.]
 [2000.   16.   15.  240.  240.  240. 1000.    0.    0.]
 [   0.   32.   15.   30.   75.  120.  704.    0.   60.]
 [  36.   60.   33.   66.  165.  264.  720.    0.   40.]
 [2000.   18.   50.  240.  240.  240. 1000.    0.    0.]
 [  32.   32.   71.  240.  240.  240.  704.  200.   90.]
 [  20.   20.   33.  240.  240.  240.  720.  400.  150.]
 [  36.   36.   30.  240.  240.  240.  720.  300.  120.]]


Extraemos la informacion de las siguientes matrices:

Stock: Strock inicial

BOX: Piezs por caja

Prices: Coste de carda referencia

Matrix_Decision: matriz para la toma de decision de qué es lo que producimos

T: Tiempos de ciclo

In [316]:
#extraigo el stock inicial
Stock_ini = np.expand_dims(np.copy(Almacen[:,0]), axis = 0)
#Extraigo las piezas por caja
BOX = np.expand_dims(np.copy(Almacen[:,1]), axis = 0)
#extraigo los precios de cada referencia
Prices = np.expand_dims(np.copy(Almacen[:,2]), axis = 0)
#Extraigo la matriz de decisiones
Matrix_decision = np.copy(Almacen[:,3:5])
#Extraigo los valores del lote_m,inimo y máximo
Matrix_lote_min = np.expand_dims(np.copy(Almacen[:,7]),axis = 0)
Matrix_lote_max = np.expand_dims(np.copy(Almacen[:,6]),axis = 0)
#Extraigo los tiempos de ciclo
T = np.expand_dims(np.copy(Almacen[:,8]), axis = 0)
T_current = np.expand_dims(np.copy(Almacen[:,8]), axis = 0)
T_current [mask_compras] = -1


##Procedimiento de cálculo de las demandas internas

In [317]:
#Voy incremetnando la matriz de demandas a medida que aumenta el nicel de fabricación
def Calcula_necesidades_por_nivel(DC,BOM_entrada,Profundidad):
  for profIndex in range(Profundidad-1):
    DC[:,profIndex+1] = np.dot(BOM_entrada.T,DC[:,profIndex])
  return(DC)

In [318]:
#Voy incremetnando la matriz de demandas a medida que aumenta el nicel de fabricación
def Calcula_necesidades(DC, BOM_entrada, Profundidad):
  DC = Calcula_necesidades_por_nivel(BOM_entrada,Profundidad)
  return(DC.sum(axis = 1))

##Tabla de cálculo de stocks mínimos
Esta table esvirá para conocer si disponemos de stock suficiente para trabajar en el proceso considerando que queremos realizar el stock mínimo.


In [319]:
Factor_BOM = np.copy(BOM)
Factor_BOM[Factor_BOM > 0] = 1
#Factor_BOM = np.divide(np.multiply(Factor_BOM, T.T),T)
Factor_BOM2 = np.multiply(Factor_BOM, T.T)
Factor_BOM = np.divide(Factor_BOM2,T.T)
#Elimino los NAN y los INF
Factor_BOM = np.where(np.isnan(Factor_BOM),0,Factor_BOM)
Factor_BOM = np.where(np.isinf(Factor_BOM),0,Factor_BOM)
Demandas_cliente = np.zeros((known_references,Profundidad))
Demandas_cliente [:,0] = Matrix_lote_min
Sumando1 = Calcula_necesidades_por_nivel(Demandas_cliente, BOM, Profundidad).astype(int)
Sumando2 = Calcula_necesidades_por_nivel(Demandas_cliente, Factor_BOM, Profundidad).astype(int)
Matrix_lote_min_internos = Sumando1 - Sumando2
df4 = pd.DataFrame(Matrix_lote_min_internos, index=REF.keys(), columns= np.arange(0,15))
print(df4)
Stock_minimo = Matrix_lote_min_internos.sum(axis = 1)
print(Stock_minimo)


     0    1    2    3   4   5   6   7   8   9   10  11  12  13  14
AAA   0    0    0  200   0   0   0   0   0   0   0   0   0   0   0
AMA   0    0 -833    0   0   0   0   0   0   0   0   0   0   0   0
BAJ   0    0  200    0   0   0   0   0   0   0   0   0   0   0   0
AAB   0 -180    0    0   0   0   0   0   0   0   0   0   0   0   0
CAT   0  200    0    0   0   0   0   0   0   0   0   0   0   0   0
AAE   0    0    0    0   0   0   0   0   0   0   0   0   0   0   0
BAC   0    0    0    0   0   0   0   0   0   0   0   0   0   0   0
CAR   0    0    0    0   0   0   0   0   0   0   0   0   0   0   0
AAF   0    0    0    0   0   0   0   0   0   0   0   0   0   0   0
BAD   0    0    0    0   0   0   0   0   0   0   0   0   0   0   0
CAV   0    0    0    0   0   0   0   0   0   0   0   0   0   0   0
AAC   0  -30    0    0   0   0   0   0   0   0   0   0   0   0   0
CAB   0    0    0    0   0   0   0   0   0   0   0   0   0   0   0
CAX   0    0    0    0   0   0   0   0   0   0   0   0   0   0

<ipython-input-319-77bdf6396c58>:5: RuntimeWarning:

invalid value encountered in true_divide



# **Inicio de modelado de simulación**
El modelo de la simulación requiere que se definan:
1. Descipción de los componentes del sistema (ver zona anterior)
2. El reloj de la simulación (arreglo de tiempos) se considera trabajar por semanas de 6 días 0-lunes -- 5 sábado
3. Definición del estado inicial del sistema
4. El modelo para el cambio de estado
5. Procedimiento para adelantar el reloj
Como resultado del sistema se provee:
1. Condición inicial
2. Listado de eventos (aleatorios + paramétricos)
3. Resultado obtenido de la ejecución en parámetros evaluables de la efectividad del resultado.

In [320]:
def Actualiza_stock(mask_finished_part, Stock_ini):
  produced = np.zeros((known_references,1))
  produced[mask_finished_part.T] = 1
  #print(produced)
  #print(Stock_ini)
  Stock_ini = Stock_ini + produced.T
  Stock_ini = Stock_ini - np.dot(BOM.T, produced).T
  #print(Stock_ini)
  #print(np.dot(BOM.T, produced).T)
  #input("Press Enter to continue...")
  return(Stock_ini)

In [321]:
#Funcion para el cálculo de piezas producidas en un tiempo determinado en segundos.
def Produce( seconds, T_current,  Stock_ini, T):
  i = seconds
  while i > 0 :
    mask_only_positives = T_current > 0
    Value = np.min(T_current[T_current > 0])
    i = i - min (i , Value)
    T_current = T_current - np.ones(known_references) * min (i , Value)
    mask_finished_part = T_current == np.zeros((known_references))
    if (mask_finished_part.any() == True):
      Stock_ini = Actualiza_stock(mask_finished_part, Stock_ini)
      T_current[mask_finished_part] = T[mask_finished_part]
    is_negative = np.any(Stock_ini <= 0)
    if is_negative:
      break
  return(T_current, Stock_ini)


In [328]:
# Set the matplotlib backend
%matplotlib inline
# Create the bar graph
Almacen_puntual_df = Stock_minimo
Almacen_puntual_df [DC [:,0] > 0] = 1
Almacen_puntual_df [Almacen_puntual_df <= 0] = 500
Almacen_puntual_df = pd.DataFrame(np.expand_dims(Almacen_puntual_df, axis = 0), columns = REF.keys())
#pongo un tiepo de visualizacion en minutos
dias = 2
Tiempo_visualizacion = 300
T_current = T
T_current [T == 0] = -1
Ciclos_visualizacion = dias * 24 * 3600 / Tiempo_visualizacion
Ciclos_visualizacion = int(Ciclos_visualizacion)
#print(Ciclos_visualizacion)
for i in range (1, Ciclos_visualizacion):
  T_current,  Almacen_puntual_df = Produce( 900, T_current,  Almacen_puntual_df, T)
  mask_finished_part = T_current == 0
  Almacen_puntual_df = Actualiza_stock(mask_finished_part, Almacen_puntual_df)
  #Update the bar heights
  current_df = pd.DataFrame(Almacen_puntual_df, columns = REF.keys())
  Almacen_puntual_df = pd.concat([Almacen_puntual_df, current_df])
  is_negative = np.any(Almacen_puntual_df <= 0)
  if is_negative:
    break
print(Almacen_puntual_df)

    AAA     AMA   BAJ    AAB    CAT   AAE    BAC    CAR    AAF    BAD    CAV  \
0   0.0  444.13  34.0  380.6  322.0  97.0  445.0  702.0  427.0  408.0  684.0   
0   0.0  444.13  34.0  380.6  322.0  97.0  445.0  702.0  427.0  408.0  684.0   
0   0.0  444.13  34.0  380.6  322.0  97.0  445.0  702.0  427.0  408.0  684.0   
0   0.0  444.13  34.0  380.6  322.0  97.0  445.0  702.0  427.0  408.0  684.0   
0   0.0  444.13  34.0  380.6  322.0  97.0  445.0  702.0  427.0  408.0  684.0   
..  ...     ...   ...    ...    ...   ...    ...    ...    ...    ...    ...   
0   0.0  444.13  34.0  380.6  322.0  97.0  445.0  702.0  427.0  408.0  684.0   
0   0.0  444.13  34.0  380.6  322.0  97.0  445.0  702.0  427.0  408.0  684.0   
0   0.0  444.13  34.0  380.6  322.0  97.0  445.0  702.0  427.0  408.0  684.0   
0   0.0  444.13  34.0  380.6  322.0  97.0  445.0  702.0  427.0  408.0  684.0   
0   0.0  444.13  34.0  380.6  322.0  97.0  445.0  702.0  427.0  408.0  684.0   

      AAC    CAB   CAX   CAH  
0   418.

In [329]:
#printing results
import bar_chart_race as bcr
import plotly
columns = REF.keys()
data = Almacen_puntual_df[columns]
fig = px.bar(Almacen)
bcr.bar_chart_race(df = Almacen_puntual_df,filename = None, fixed_order=True, figsize = (3.5 , 3), title =" Stock por referencia")

/usr/local/lib/python3.10/dist-packages/bar_chart_race/_make_chart.py:286: UserWarning:

FixedFormatter should only be used together with FixedLocator

/usr/local/lib/python3.10/dist-packages/bar_chart_race/_make_chart.py:287: UserWarning:

FixedFormatter should only be used together with FixedLocator



Exception: ignored

# Grabado de datos en un EXCEL

*Reloj de la simulación* - se establece como una matriz horarira de 24 x 6 días (programación semanal)

In [324]:
Calendario_Sim = np.zeros((24,6))
print(Calendario_Sim)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


*Definir el estado inicial* - Estado de las máquinas y almacenamiento

Se definie un Coste de referencia para el calculo del coste de material en el almacén

In [325]:
Process_status = np.zeros((len(DICMAQ),1))
Process_status[DICMAQ["Inyeccion"]] = REF["5"]
Process_status[DICMAQ["Mecanizado"]] = REF["8"]
STOCK = {"Referencia":0,"Volumen":1,"Coste":2}
Reference_cost = np.zeros((len(REF),1))
Reference_cost[REF["1"]] = 12.8
Reference_cost[REF["2"]] = 23.7
Reference_cost[REF["3"]] = 18.4
Reference_cost[REF["4"]] = 6.2
Reference_cost[REF["5"]] = 50.1
Reference_cost[REF["6"]] = 80.2
Reference_cost[REF["7"]] = 67.4
Reference_cost[REF["8"]] = 73.9

Almacen_MP = np.zeros((3,known_references))
Almacen_MP[STOCK["Referencia"],REF["1"]] = REF["1"] # Aunque sea redundante, es solo un tema de legibilidad
Almacen_MP[STOCK["Volumen"],REF["1"]] = 5
Almacen_MP[STOCK["Coste"],REF["1"]] = Almacen_MP[STOCK["Volumen"],0] * Reference_cost[REF["1"]]

Almacen_MP[STOCK["Referencia"],REF["2"]] = REF["2"] # Aunque sea redundante, es solo un tema de legibilidad
Almacen_MP[STOCK["Volumen"],REF["2"]] = 7
Almacen_MP[STOCK["Coste"],REF["2"]] = Almacen_MP[STOCK["Volumen"],0] * Reference_cost[REF["2"]]

Almacen_MP[STOCK["Referencia"],REF["3"]] = REF["3"] # Aunque sea redundante, es solo un tema de legibilidad
Almacen_MP[STOCK["Volumen"],REF["3"]] = 1
Almacen_MP[STOCK["Coste"],REF["3"]] = Almacen_MP[STOCK["Volumen"],0] * Reference_cost[REF["3"]]

Almacen_MP[STOCK["Referencia"],REF["4"]] = REF["4"] # Aunque sea redundante, es solo un tema de legibilidad
Almacen_MP[STOCK["Volumen"],REF["4"]] = 2
Almacen_MP[STOCK["Coste"],REF["4"]] = Almacen_MP[STOCK["Volumen"],0] * Reference_cost[REF["4"]]


# Notas Alf - Borrar
#MAQ_TRANS_TIEMPO[2,TRANS["Maquina"]] = DICMAQ["Inyeccion"] / "Mecanizado"
#MAQ_TRANS_TIEMPO[2,TRANS["Origen"]] = REF["5"]             / REF["7"]
#MAQ_TRANS_TIEMPO[2,TRANS["Destino"]] = REF["6"]            / REF["8"]

print("Reference costs:",Reference_cost,"\n")
print("Current status of RowMat Stock\n",Almacen_MP)
print("Process Current State: ",Process_status)

KeyError: ignored

In [ ]:
from pandas.compat import numpy
# Production diary - this is an array where
# 0. Timeframe recorded
# 1. Stock status
sim_start = dt.datetime(year=2023, month=6, day=23, hour=0, minute=0, second=0)
step = timedelta(minutes=15)
sim_frame = sim_start
sim_log = []
print(Stock_ini)
for i in range(0,12):
  log_register = []
  print("Frame [",i,"] ",sim_frame)
  sim_frame += step
  log_register.append(sim_frame.strftime("%Y-%m-%d %H:%M:%S"))
  log_register.append(Stock_ini)
  #sim_log.append(sim_frame.strftime("%Y-%m-%d %H:%M:%S"))
  sim_log.append(log_register)

sheet_name = 'ProductionDiary'
sheet.clear()
sheet = AAM_workbook.worksheet(title=sheet_name)
cells = []
for i, log_entry in enumerate(sim_log):
  cells.append(Cell(row=i+2, col=1, value=log_entry[0]))
  for j, stock_entry in enumerate(np.array(log_entry[1])):
    for k, stock_entry2 in enumerate(stock_entry):
      cells.append(Cell(row=i+2, col=k+2, value=stock_entry[k]))

sheet.update_cells(cells)
#print("SIM Log:\n",sim_log)


In [ ]:
#Graficar resultados